In [49]:
from pymongo import MongoClient
client = MongoClient()
nvd_database = client['vulnerability']['nvd']
nvd_results = nvd_database.find({}, {
    'cve.CVE_data_meta.ID': 1,
    'configurations': 1,
    '_id': 0,
})

In [50]:
from collections import defaultdict
d = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(set))))

In [51]:
from collections import defaultdict
from itertools import chain

def yield_cpe_pack(results):
    for r in results:
        cve_id = r['cve']['CVE_data_meta']['ID']
        for node in r['configurations']['nodes']:
            for cpe_match in extract_cpe_matches(node):
                yield cve_id, cpe_match
                
def extract_cpe_matches(node):
    if 'cpe_match' in node:
        return node['cpe_match']
    if 'children' in node:
        return chain(*[extract_cpe_matches(_) for _ in node['children']])
    return []

In [52]:
def get_product_version_pack(cpe_match, product_version):
    a_excluding = None
    a_including = None
    b_including = None
    b_excluding = None
    if product_version:
        a_including = product_version
        b_including = product_version
    if 'versionStartExcluding' in cpe_match:
        a_excluding = cpe_match['versionStartExcluding']
    if 'versionStartIncluding' in cpe_match:
        a_including = cpe_match['versionStartIncluding'] 
    if 'versionEndIncluding' in cpe_match:
        b_including = cpe_match['versionEndIncluding']
    if 'versionEndExcluding' in cpe_match:
        b_excluding = cpe_match['versionEndExcluding']
    return a_excluding, a_including, b_including, b_excluding

In [53]:
for cve_id, cpe_match in yield_cpe_pack(nvd_results):
    cpe_pack = cpe_match['cpe23Uri'].split(':')[2:6]
    component_type = cpe_pack[0]
    vendor_name = cpe_pack[1]
    product_name = cpe_pack[2]
    product_version = cpe_pack[3]
    product_version_pack = get_product_version_pack(
        cpe_match, product_version)
    d[component_type][vendor_name][product_name][
        product_version_pack].add(cve_id)

In [54]:
def get_dictionary(d):
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = get_dictionary(v)
    return dict(d)

d = get_dictionary(d)

In [55]:
d.keys()

dict_keys(['o', 'a', 'h'])

In [56]:
list(d['h'].keys())[:3]

['nec', 'lucent', 'ibm']

In [57]:
d['h']['selinc'].keys()

dict_keys(['sel-2241', 'sel-3505', 'sel-3530', 'sel-3530-4', 'sel-3620', 'sel-3622'])

In [58]:
d['h']['selinc']['sel-3620']

{(None, '-', '-', None): {'CVE-2017-7928'}}

In [65]:
d['o']['selinc']['sel-3620_firmware']

{(None, 'r202', 'r202', None): {'CVE-2017-7928'},
 (None, 'r203', 'r203', None): {'CVE-2017-7928'},
 (None, 'r203-v', 'r203-v', None): {'CVE-2017-7928'},
 (None, 'r203-v1', 'r203-v1', None): {'CVE-2017-7928'},
 (None, 'r204', 'r204', None): {'CVE-2017-7928'},
 (None, 'r204-v1', 'r204-v1', None): {'CVE-2017-7928'}}

In [59]:
selected_nvd_results = nvd_database.find({'cve.CVE_data_meta.ID': 'CVE-2017-7928'}, {
    'cve.CVE_data_meta.ID': 1,
    'configurations': 1,
    '_id': 0,
})
selected_cpe_packs = list(yield_cpe_pack(selected_nvd_results))

In [60]:
for cve_id, cpe_match in selected_cpe_packs:
    cpe_pack = cpe_match['cpe23Uri'].split(':')[2:6]
    component_type = cpe_pack[0]
    vendor_name = cpe_pack[1]
    product_name = cpe_pack[2]
    product_version = cpe_pack[3]
    if component_type == 'h':
        break

In [61]:
cpe_match

{'cpe23Uri': 'cpe:2.3:h:selinc:sel-3620:-:*:*:*:*:*:*:*', 'vulnerable': False}

In [62]:
product_version

'-'